In [1]:
pip install transformers torch kaggle json

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Note: you may need to restart the kernel to use updated packages.


In [2]:
from huggingface_hub import login
login('Token here')  # Replace 'your_huggingface_token' with your actual token


In [6]:
import json
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load JSON files
with open("/kaggle/input/newdata/Inefficient_reasoning.json", "r") as f:
    reasoning_data = json.load(f)

with open("/kaggle/input/newdata/balanced_samples.json", "r") as f:
    codes_data = json.load(f)

# Convert reasoning_data into a dictionary for easy lookup
reasoning_dict = {item["question_id"]: item for item in reasoning_data}

# Initialize Meta Llama model and tokenizer
MODEL_NAME = "meta-llama/Llama-3.2-1B"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16).to(device)

# Function to process and generate optimized code
def get_optimized_code():
    optimized_solutions = []
    api_calls = 0  # Track API call count

    for index, code_entry in enumerate(codes_data, start=1):
        qid = code_entry["question_id"]

        # Check if question_id exists in reasoning_dict
        if qid not in reasoning_dict:
            print(f"Skipping question_id {qid} (not found in reasoning data)")
            continue

        # Ensure "reason" key exists
        if "reason" not in reasoning_dict[qid]:
            print(f"Skipping question_id {qid} (missing 'reason' key)")
            continue

        # Ensure "runtime_inefficient_codes" exists and is not empty
        if "runtime_inefficient_codes" not in code_entry or not code_entry["runtime_inefficient_codes"]:
            print(f"Skipping question_id {qid} (missing or empty 'runtime_inefficient_codes')")
            continue

        # Get reasoning and first inefficient code
        reasoning = reasoning_dict[qid]["reason"]
        inefficient_code = code_entry["runtime_inefficient_codes"][0]["code"]

        # Create prompt
        prompt = f"""
        The following Python code seems to be inefficient:
        
        ```python
        {inefficient_code}
        ```
        
        Reason for inefficiency:
        {reasoning}
        
        Please rewrite the code based on the provided reason, and only return the corrected code, no explanation.
        Use Python 3.8 syntax, and ensure the entire solution is enclosed in a `class Solution`, 
        maintaining the same function name as in the input code.
        Directly start with the code which can be put in the executor and no statement or word before that.
        """

        try:
            # Tokenize the input prompt
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)

            # Generate content using Meta Llama model
            outputs = model.generate(
                input_ids=inputs["input_ids"], 
                attention_mask=inputs["attention_mask"],
                max_new_tokens=250,  # Adjust as per the required length of output
                min_length=50,  # Ensure enough content in response
                temperature=0.3,  # Keep the output deterministic
                num_beams=5,  # Beam search to improve output quality
                repetition_penalty=1.2,  # Avoid repetitive content
                early_stopping=True
            )

            # Decode the generated output
            generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Extract the generated code part (remove prompt)
            if "```python" in generated_code:
                generated_code = generated_code.split("```python")[-1].split("```")[0].strip()

            if generated_code:
                optimized_solutions.append({
                    "question_id": qid,
                    "optimized_code": generated_code.strip()
                })
                print(f"✅ Successfully optimized question_id {qid} ({index}/{len(codes_data)})")
            else:
                print(f"⚠️ Warning: Empty response for question_id {qid}")

        except Exception as e:
            print(f"❌ Error processing question_id {qid}: {e}")
            continue  # Skip this question and proceed

        # Increment API call counter
        api_calls += 1

        # # Sleep after every 14 API calls to avoid exhaustion
        # if api_calls % 14 == 0:
        #     print("⏳ API limit reached. Sleeping for 60 seconds...")
        #     time.sleep(60)

    return optimized_solutions

# Run the function
optimized_results = get_optimized_code()

# Save the results
with open("optimized_codes.json", "w") as f:
    json.dump(optimized_results, f, indent=4)

print("🎉 Optimized code has been saved to optimized_codes.json")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


✅ Successfully optimized question_id 131 (1/200)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


✅ Successfully optimized question_id 132 (2/200)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

✅ Successfully optimized question_id 855 (195/200)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


✅ Successfully optimized question_id 856 (196/200)
Skipping question_id 564 (missing 'reason' key)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


✅ Successfully optimized question_id 552 (198/200)
Skipping question_id 938 (missing 'reason' key)
✅ Successfully optimized question_id 939 (200/200)
🎉 Optimized code has been saved to optimized_codes.json


NEW

In [14]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load JSON files
with open("/kaggle/input/newdata/Inefficient_reasoning.json", "r") as f:
    reasoning_data = json.load(f)

with open("/kaggle/input/newdata/balanced_samples.json", "r") as f:
    codes_data = json.load(f)

# Convert reasoning_data into a dictionary for easy lookup
reasoning_dict = {item["question_id"]: item for item in reasoning_data}

# Initialize Meta Llama model and tokenizer
MODEL_NAME = "meta-llama/Llama-3.2-1B"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16).to(device)

# Function to process and generate optimized code
def get_optimized_code():
    optimized_solutions = []
    api_calls = 0  # Track API call count

    for index, code_entry in enumerate(codes_data, start=1):
        qid = code_entry["question_id"]

        # Check if question_id exists in reasoning_dict
        if qid not in reasoning_dict:
            print(f"Skipping question_id {qid} (not found in reasoning data)")
            continue

        # Ensure "reason" key exists
        if "reason" not in reasoning_dict[qid]:
            print(f"Skipping question_id {qid} (missing 'reason' key)")
            continue

        # Ensure "runtime_inefficient_codes" exists and is not empty
        if "runtime_inefficient_codes" not in code_entry or not code_entry["runtime_inefficient_codes"]:
            print(f"Skipping question_id {qid} (missing or empty 'runtime_inefficient_codes')")
            continue

        # Get reasoning and first inefficient code
        reasoning = reasoning_dict[qid]["reason"]
        inefficient_code = code_entry["runtime_inefficient_codes"][0]["code"]

        # Create prompt
        prompt = f"""
        The following Python code is inefficient. Based on the reason provided below, please rewrite and optimize the code:

        Inefficient Code:
        ```python
        {inefficient_code}
        
        Reason for inefficiency:
        {reasoning}
        
        Please return only the optimized code (no explanation), ensuring that it performs better based on the given reasoning. Ensure the entire solution is enclosed in a class Solution, maintaining the same function name as in the input code. Use Python 3.8 syntax and optimize the code's efficiency.
        """

        try:
            # Tokenize the input prompt
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)

            # Generate content using Meta Llama model
            outputs = model.generate(
                input_ids=inputs["input_ids"], 
                attention_mask=inputs["attention_mask"],
                max_new_tokens=500,  # Increase max tokens for longer output
                min_length=100,  # Ensure enough content in response
                temperature=0.7,  # Increased temperature for more diverse output
                num_beams=5,  # Beam search to improve output quality
                repetition_penalty=1.2,  # Avoid repetitive content
                early_stopping=True
            )

            # Decode the generated output
            generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Post-process the output to extract code block
            # Extract the code after '```python' and before the closing '```'
            if "```python" in generated_code:
                generated_code = generated_code.split("```python")[-1].split("```")[0].strip()

            if generated_code:
                optimized_solutions.append({
                    "question_id": qid,
                    "optimized_code": generated_code.strip()
                })
                print(f"✅ Successfully optimized question_id {qid} ({index}/{len(codes_data)})")
            else:
                print(f"⚠️ Warning: Empty response for question_id {qid}")

        except Exception as e:
            print(f"❌ Error processing question_id {qid}: {e}")
            continue  # Skip this question and proceed

        # Increment API call counter
        api_calls += 1

    return optimized_solutions

# Run the function
optimized_results = get_optimized_code()

# Save the results
with open("optimized_codes.json", "w") as f:
    json.dump(optimized_results, f, indent=4)

print("🎉 Optimized code has been saved to optimized_codes.json")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


✅ Successfully optimized question_id 131 (1/200)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


✅ Successfully optimized question_id 132 (2/200)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

✅ Successfully optimized question_id 855 (195/200)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


✅ Successfully optimized question_id 856 (196/200)
Skipping question_id 564 (missing 'reason' key)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


✅ Successfully optimized question_id 552 (198/200)
Skipping question_id 938 (missing 'reason' key)
✅ Successfully optimized question_id 939 (200/200)
🎉 Optimized code has been saved to optimized_codes.json
